# Keras MiniVGGNet example on flowers17

In [6]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras_example.preprocessing.imagetoarrayprocessor import ImageToArrayPreprocessor
from keras_example.preprocessing.aspectawareprocessor import AspectAwareProcessor
from keras_example.datasets.simpledatasetloader import SimpleDatasetLoader
from keras_example.nn.conv.minivggnet import MiniVGGNet

from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [5]:
DATASET = ''

In [ ]:
print('[INFO] loading images...')
imagePaths = list(paths.list_images(DATASET))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

In [ ]:
# Initialize preprocessors
aap = AspectAwareProcessor(64,64)
iap = ImageToArrayPreprocessor()


sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") /  255.0

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
test_size=0.25, random_state=42)

trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [7]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=0.05)
model = MiniVGGNet.build(width=64, height=64, depth=3,
classes=len(classNames))
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])
# train the network
print("[INFO] training network...")
BATCH_SIZE = 32
EPOCHS = 100

H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BATCH_SIZE),validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BATCH_SIZE, epochs=EPOCHS, verbose=1)

[INFO] compiling model...


/home/newton/miniconda3/envs/expmlenv/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


NameError: name 'classNames' is not defined

In [ ]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
predictions.argmax(axis=1), target_names=classNames))

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 100), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 100), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 100), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, 100), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()